# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.power import TTestIndPower, TTestPower
%matplotlib inline

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
df = pd.read_csv('homepage_actions.csv', date_parser="timestamp")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df.shape

(8188, 4)

In [5]:
df.loc[df.duplicated(subset="id", keep=False)]

,timestamp,id,group,action
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
11,2016-09-24 21:05:15.348935,601714,experiment,view
12,2016-09-24 21:06:27.553057,601714,experiment,click
14,2016-09-24 21:29:19.766467,487634,experiment,view
15,2016-09-24 21:30:02.739756,487634,experiment,click
16,2016-09-24 23:01:08.713402,468601,experiment,view
17,2016-09-24 23:01:12.108316,468601,experiment,click
20,2016-09-25 00:00:47.700734,555973,experiment,view
21,2016-09-25 00:01:47.933853,555973,experiment,click


In [14]:
len(df.id.unique())

6328

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [5]:
df.group.unique()

array(['experiment', 'control'], dtype=object)

In [6]:
df.action.unique()

array(['view', 'click'], dtype=object)

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [18]:
pivot = df.pivot_table(values='id', index='group', columns='action', aggfunc='count')
pivot["probability"] = pivot.click / pivot.view
pivot["std"] = np.sqrt(pivot.probability * (1 - pivot.probability) / pivot.view)
pivot

action,click,view,probability,std
group,,,,
control,932,3332,0.279712,0.007776
experiment,928,2996,0.309746,0.008448


In [19]:
(pivot.loc['control','probability'] - pivot.loc['experiment','probability']) / np.sqrt((pivot.loc['control','std'] ** 2 * (pivot.loc['control','view'] - 1) + pivot.loc['experiment','std'] ** 2 * (pivot.loc['experiment','view'] - 1))/(pivot.loc['control','view'] - 1 + pivot.loc['experiment','view'] - 1))
                                                                                       

-3.7075272738832257

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [20]:
v1 = pivot.loc['experiment', 'view'] * pivot.loc['control', 'probability']
v1

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [25]:
v2 = pivot.loc['experiment', 'view'] * pivot.loc['experiment', 'probability'] * (1 - pivot.loc['experiment', 'probability'])
np.sqrt(v2)

25.30919609963171

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [24]:
(pivot.loc['experiment', 'click'] - v1) / np.sqrt(v2)

3.555355647136508

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.